In [1]:
import os 
os.path.abspath(r"C:\Users\Derrick\Desktop\Traffic Detection Model")
import cv2
import numpy as np
import time

# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


In [4]:
# Loading image
cap = cv2.VideoCapture('ThikaRoad.mp4')
#instantiate a variable 'p' to keep count of vehicles
p = 0 
font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0
while True:
    _, frame = cap.read()
    frame_id += 1
    height, width, channels = frame.shape
    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # initialize our lists of detected bounding boxes, confidences, and class IDs, respectively
    class_ids = []
    confidences = []
    boxes = []
    # loop over each of the layer outputs
    for out in outs:
        # loop over each of the detections
        for detection in out:
            # extract the class ID and confidence 
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # filter out weak predictions
            if confidence > 0.7:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # update our list of bounding box coordinates, confidences, and class IDs
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # apply non-maxima suppression to suppress weak, overlapping bounding boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)
    
    #detecting Vehicles
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label=="car":
                p=p+1
            elif label=="truck":
                p=p+1
            elif label=="bus":
                p=p+1
            elif label == "motor bike":
                p=p+1
            else:
                continue
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + ':' + str(p) , (x, y + 30), font, 1, color, 1)
    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "Frame Per Second: " + str(round(fps, 2)), (10, 50), font, 1, (0, 0, 0), 1)
    cv2.putText(frame, "Count: " + str(p), (10, 80), font, 1, (0, 0, 0), 1)
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        break
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'